In [1]:
# !pip install Beautifulsoup4
# !pip install selenium
# !pip install webdriver_manager
# !pip install pandas

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.request
import time
import re

import pandas as pd
import pprint

from bs4 import BeautifulSoup

# 크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메세지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

### 다나와 상세페이지에서 테이블 내용을 불러오기

In [3]:
user_agent ="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
chrome_options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(options=chrome_options)

In [24]:
# 크롤링 참고 (헤더)

header = ["name", "price", "link", "company", "product_seq", "image", "generation",
          "capacity", "clock", "timing", "number", "ecc", "xmp", "expo", "heatsink",
          "heatsink_color", "led", "led_color", "reg_date", "bookmark"]


ssd = dict()

for col in header:
    ssd[col] = []

In [25]:
save_file = False  # 파일 저장 여부
waiting_sec = 3

# seq 파일 읽기
# category = 112747  # ssd
# category = 112751  # 메인보드
category = 112752  # 램
# category = 112753  # 그래픽카드
# category = 112760  # SSD
# category = 112763  # HDD
# category = 112775  # 케이스
# category = 112777  # 파워
# category = 11236855  # 쿨러

seq_file = f'./seq_{category}.csv'
seq_df = pd.read_csv(seq_file)
len_seq = len(seq_df)
print(len_seq)

# 비트마스킹
m2i_bitmask = ['SATA', 'NVMe', 'PCIe', 'PCIe4.0', 'PCIe5.0']
m2f_bitmask = ['2230', '2242', '2260', '2280', '2580', '22110', '25110']
wl_bitmask = ['무선 LAN', '블루투스', 'M.2 Key-E(모듈별매)']
# 딕셔너리 초기화
ssd = dict()
ssd_pci = dict()

for col in header:
    ssd[col] = []

1000


In [26]:
# 크롤링 참고 (크롤링, 딕셔너리로 저장)

# chrome_options = Options()

user_agent ="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
chrome_options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(options=chrome_options)


# 크롤링 시작
ntime = 1
start = time.time()

for seq in seq_df.seq:
    url = f'https://prod.danawa.com/info/?pcode={seq}&cate={category}'
    driver.get(url)
    driver.implicitly_wait(10)

    time.sleep(waiting_sec)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    spec_tbl = soup.select("#productDescriptionArea > div > div.prod_spec > table > tbody")
    spec_tbl_tit = soup.select("#productDescriptionArea > div > div.prod_spec > table > tbody > tr > th.tit")
    spec_tbl_dsc = soup.select("#productDescriptionArea > div > div.prod_spec > table > tbody > tr > td.dsc")
    name = soup.select_one("#blog_content > div.summary_info > div.top_summary > h3 > span")
    price = soup.select_one("#blog_content > div.summary_info > div.detail_summary > div.summary_left > div.lowest_area > div.lowest_top > div.row.lowest_price > span.lwst_prc > a > em")
    image_url = soup.select_one("#baseImage")
    
    tbl = zip(spec_tbl_tit, spec_tbl_dsc)

    info = dict()
    info['제품명'] = name.get_text()
    info['최저가'] = price.get_text() if price else None
    info['링크'] = url
    info['이미지'] = image_url['src']
    for tit, dsc in tbl:
        if tit.get_text() in ['안전확인인증','적합성평가인증']:
            continue
        text = dsc.get_text()
        text = text.replace('\n','')
        text = text.replace('\t','')
        text = text.replace('(제조사 웹사이트 바로가기)','')
        info[tit.get_text()] = text.strip()
    
    # pprint.pprint(info)
    
    if info.get('사용 장치')!='데스크탑용':
        ntime+=1
        continue
    
    m2i_bit = 0
    m2f_bit = 0
    wl_bit = 0
    wls = 0
    
    
    
    ram['name'].append(name.get_text()[:name.get_text().index('(')].rstrip())
    ram['price'].append(price.get_text().replace(',','') if price else None)
    ram['link'].append(url)
    ram['company'].append(info.get('제조회사'))
    ram['product_seq'].append(seq)
    ram['image'].append(image_url.get('src'))
    
    ram['generation'].append(info.get('제품 분류'))
    
    ram['capacity'].append(info.get('메모리 용량'))

    ram['clock'].append(info.get('동작클럭(대역폭)'))
    ram['timing'].append(info.get('램타이밍'))

    ram['number'].append(info.get('램개수'))
    
    # ecc
    if 'ECC' in info.keys():
        ram['ecc'].append(1)
    elif '온다이ECC' in info.keys():
        ram['ecc'].append(2)
    else:
        ram['ecc'].append(None)
        
    # XMP
    if 'XMP' in info.keys():
        ram['xmp'].append(1)
    elif 'XMP3.0' in info.keys():
        ram['xmp'].append(2)
    else:
        ram['xmp'].append(None)
        
    ram['expo'].append(info.get('EXPO'))
    ram['heatsink'].append(1 if info.get('히트싱크')=='방열판' else 0)
    
    ram['heatsink_color'].append(info.get('방열판 색상'))
    
    ram['led'].append(1 if info.get('LED 라이트') else 0)
    ram['led_color'].append(info.get('LED색상'))
                          
    ram['reg_date'].append(re.sub('(년| |월)', '', info.get('등록년월')))
    ram['bookmark'].append(info.get('북마크'))
    
    
    print(f'\r{ntime} / {len_seq} : {int(ntime/len_seq*100)}%',end='')
    # if ntime==20:
    #     break
    ntime+=1

end = time.time()
print(f"\n걸린 시간: {end-start:.2f}sec")
driver.quit()
# 크롤링 끝

1000 / 1000 : 100%
걸린 시간: 6247.27sec


In [30]:
ram_df = pd.DataFrame(ram)
ram_df

name   price  \
0                                삼성전자 DDR4-3200   39810   
1             TeamGroup DDR5-5600 CL46 Elite 서린   54872   
2    마이크론 Crucial DDR5-5600 CL46 PRO 패키지 대원씨티에스  111680   
3            마이크론 Crucial DDR4-3200 CL22 대원씨티에스   19390   
4      OLOy DDR5-6000 CL32 BLADE RGB MIRROR 패키지  190970   
..                                          ...     ...   
858                     EKMEMORY DDR3-1066 인디핑크    None   
859                         CYNEX DDR3-1333 엠트란    None   
860                          CYNEX DDR3-1066 멜론    None   
861                         CYNEX DDR3-1066 엠트란   29128   
862                             CYNEX DDR3-1066   24366   

                                                  link    company  \
0    https://prod.danawa.com/info/?pcode=11790199&c...       삼성전자   
1    https://prod.danawa.com/info/?pcode=17535839&c...  TeamGroup   
2    https://prod.danawa.com/info/?pcode=20369066&c...       마이크론   
3    https://prod.danawa.com/info/?pcode=20369102&c...       마이크론   
4    https://prod.danawa.com/info/?pcode=17770829&c...       OLOy   
..                                                 ...        ...   
858  https://prod.danawa.com/info/?pcode=1183683&ca...   EKMEMORY   
859  https://prod.danawa.com/info/?pcode=794443&cat...      CYNEX   
860  https://prod.danawa.com/info/?pcode=793964&cat...      CYNEX   
861  https://prod.danawa.com/info/?pcode=793961&cat...      CYNEX   
862  https://prod.danawa.com/info/?pcode=793959&cat...      CYNEX   

     product_seq                                              image  \
0       11790199  //img.danawa.com/prod_img/500000/199/790/img/1...   
1       17535839  //img.danawa.com/prod_img/500000/839/535/img/1...   
2       20369066  //img.danawa.com/prod_img/500000/066/369/img/2...   
3       20369102  //img.danawa.com/prod_img/500000/102/369/img/2...   
4       17770829  //img.danawa.com/prod_img/500000/829/770/img/1...   
..           ...                                                ...   
858      1183683  //img.danawa.com/prod_img/500000/683/183/img/1...   
859       794443  //img.danawa.com/prod_img/500000/443/794/img/7...   
860       793964  //img.danawa.com/prod_img/500000/964/793/img/7...   
861       793961  //img.danawa.com/prod_img/500000/961/793/img/7...   
862       793959  //img.danawa.com/prod_img/500000/959/793/img/7...   

    generation capacity                clock         timing number  ecc  xmp  \
0         DDR4     16GB  3200MHz (PC4-25600)           None     1개  NaN  NaN   
1         DDR5     16GB  5600MHz (PC5-44800)  CL46-46-46-90     1개  2.0  NaN   
2         DDR5     32GB  5600MHz (PC5-44800)     CL46-45-45     2개  2.0  2.0   
3         DDR4      8GB  3200MHz (PC4-25600)           CL22     1개  NaN  NaN   
4         DDR5     32GB  6000MHz (PC5-48000)  CL32-38-38-76     2개  2.0  NaN   
..         ...      ...                  ...            ...    ...  ...  ...   
858       DDR3      2GB   1066MHz (PC3-8500)           None     1개  NaN  NaN   
859       DDR3      2GB  1333MHz (PC3-10600)            CL9     1개  NaN  NaN   
860       DDR3      2GB   1066MHz (PC3-8500)            CL7     1개  NaN  NaN   
861       DDR3      2GB   1066MHz (PC3-8500)            CL7     1개  NaN  NaN   
862       DDR3      2GB   1066MHz (PC3-8500)            CL7     1개  NaN  NaN   

     expo  heatsink heatsink_color  led led_color reg_date bookmark  
0    None         0           None    0      None   202007     None  
1    None         0           None    0      None   202207     None  
2       ○         1             블랙    0      None   202305     None  
3    None         0           None    0      None   202001     None  
4    None         1         실버/그레이    1       RGB   202209     None  
..    ...       ...            ...  ...       ...      ...      ...  
858  None         1             핑크    0      None   201009     None  
859  None         1             블루    0      None   200811     None  
860  None         1            옐로우    0      None   200811   

In [53]:
pd.DataFssde(ssd).to_csv("ssd.csv")

AttributeError: module 'pandas' has no attribute 'DataFssde'

In [70]:
df = pd.read_csv("./ram.csv",index_col=0)

In [71]:
def remove_str(x):
    x = str(x).replace('GB','').replace('개','')
    return x

def pre_expo(x):
    if x=='○':
        x=1
    else:
        x=0
    return x

df.capacity = df.capacity.apply(lambda x : remove_str(x))
df.number = df.number.apply(lambda x: remove_str(x))
df.expo = df.expo.apply(lambda x: pre_expo(x))
df.tail()

name    price  \
858  EKMEMORY DDR3-1066 인디핑크      NaN   
859      CYNEX DDR3-1333 엠트란      NaN   
860       CYNEX DDR3-1066 멜론      NaN   
861      CYNEX DDR3-1066 엠트란  29128.0   
862          CYNEX DDR3-1066  24366.0   

                                                  link   company  product_seq  \
858  https://prod.danawa.com/info/?pcode=1183683&ca...  EKMEMORY      1183683   
859  https://prod.danawa.com/info/?pcode=794443&cat...     CYNEX       794443   
860  https://prod.danawa.com/info/?pcode=793964&cat...     CYNEX       793964   
861  https://prod.danawa.com/info/?pcode=793961&cat...     CYNEX       793961   
862  https://prod.danawa.com/info/?pcode=793959&cat...     CYNEX       793959   

                                                 image generation capacity  \
858  //img.danawa.com/prod_img/500000/683/183/img/1...       DDR3        2   
859  //img.danawa.com/prod_img/500000/443/794/img/7...       DDR3        2   
860  //img.danawa.com/prod_img/500000/964/793/img/7...       DDR3        2   
861  //img.danawa.com/prod_img/500000/961/793/img/7...       DDR3        2   
862  //img.danawa.com/prod_img/500000/959/793/img/7...       DDR3        2   

                   clock timing number  ecc  xmp  expo  heatsink  \
858   1066MHz (PC3-8500)    NaN      1  NaN  NaN     0         1   
859  1333MHz (PC3-10600)    CL9      1  NaN  NaN     0         1   
860   1066MHz (PC3-8500)    CL7      1  NaN  NaN     0         1   
861   1066MHz (PC3-8500)    CL7      1  NaN  NaN     0         1   
862   1066MHz (PC3-8500)    CL7      1  NaN  NaN     0         0   

    heatsink_color  led led_color  reg_date  bookmark  
858             핑크    0       NaN    201009       NaN  
859             블루    0       NaN    200811       NaN  
860            옐로우    0       NaN    200811       NaN  
861             블루    0       NaN    200811       NaN  
862            NaN    0       NaN    200811       NaN

In [72]:
'○'

'○'

In [73]:
df.to_csv("ram_mod.csv",index=None)